In [1]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

#//*** create model output path if needed
output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [2]:
output_path = "tts_train_dir"
training_dir = "training"
output_path = os.path.join(os.path.join(os.getcwd(), output_path))
output_path
training_dir = os.path.join(os.path.join(os.getcwd(), training_dir)).replace("\\","/")
print(str(training_dir))
t_path = str(output_path).replace("\\","/")
t_path
print(t_path)

import os
import wave
fulldir = os.path.join(output_path,"wavs")
#//**** Get the sample rate of the first WAV file, assume all files share the same sample_rate
for file_name in os.listdir(fulldir):
    with wave.open(os.path.join(fulldir,file_name), "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        print(file_name,frame_rate)
        break


C:/Users/stonk013/KGO_Projects/tts/ama_model/training
C:/Users/stonk013/KGO_Projects/tts/ama_model/tts_train_dir
ama-01.wav 48000


In [3]:
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples


# dataset config for one of the pre-defined datasets
dataset_config = BaseDatasetConfig(
    #formatter="vctk", meta_file_train="", language="en-us", path=os.path.join(os.path.join(output_path,"wavs"))
    formatter="ljspeech", meta_file_train="metadata.csv", language="en-us", path=t_path
)

# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=False)

 | > Found 12 files in C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir


In [4]:
import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseAudioConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.configs.tacotron2_config import Tacotron2Config
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.tacotron2 import Tacotron2
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

In [5]:
audio_config = BaseAudioConfig(
    sample_rate=frame_rate,
    do_trim_silence=True,
    trim_db=60.0,
    signal_norm=False,
    mel_fmin=0.0,
    mel_fmax=8000,
    spec_gain=1.0,
    log_func="np.log",
    ref_level_db=20,
    preemphasis=0.0,
)

In [6]:
config = Tacotron2Config(  # This is the config that is saved for the future use
    audio=audio_config,
    batch_size=64,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    r=6,
    gradual_training=[[0, 6, 64], [10000, 4, 32], [50000, 3, 32], [100000, 2, 32]],
    double_decoder_consistency=True,
    epochs=1000,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(training_dir, "phoneme_cache"),
    precompute_num_workers=8,
    print_step=25,
    print_eval=True,
    mixed_precision=False,
    output_path=training_dir,
    datasets=[dataset_config],
)

In [7]:
output_path

'C:\\Users\\stonk013\\KGO_Projects\\tts\\ama_model\\tts_train_dir'

In [8]:
# init audio processor
ap = AudioProcessor(**config.audio.to_dict())

# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# If characters are not defined in the config, default characters are passed to the config
tokenizer, config = TTSTokenizer.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:48000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60.0
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Setting up Audio Processor...
 | > sample_rate:48000
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5


In [9]:
# INITIALIZE THE MODEL
# Models take a config object and a speaker manager as input
# Config defines the details of the model like the number of layers, the size of the embedding, etc.
# Speaker manager is used by multi-speaker models.
model = Tacotron2(config, ap, tokenizer, speaker_manager=None)

# init the trainer and 🚀
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)
trainer.fit()

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Num. of CPUs: 8
 | > Num. of Torch Threads: 4
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=C:/Users/stonk013/KGO_Projects/tts/ama_model/training\run-August-09-2023_10+12PM-0000000

 > Model has 47669492 parameters

 > EPOCH: 0/1000
 --> C:/Users/stonk013/KGO_Projects/tts/ama_model/training\run-August-09-2023_10+12PM-0000000

 > TRAINING (2023-08-09 22:12:48) 



 > Number of output frames: 6


> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
| > Number of instances : 12
 | > Preprocessing samples
 | > Max text length: 908
 | > Min text length: 106
 | > Avg text length: 392.8181818181818
 | 
 | > Max audio length: 4811245.0
 | > Min audio length: 645483.0
 | > Avg audio length: 2134389.1818181816
 | > Num. instances discarded samples: 1
 | > Batch group size: 0.


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:/Users/stonk013/KGO_Projects/tts/ama_model/training/run-August-09-2023_10+12PM-0000000\\trainer_0_log.txt'